In [2]:
from numpy import mat, cov, mean, hstack, multiply,sqrt,diag, \
    squeeze, ones, array, vstack, kron, zeros, eye, savez_compressed
from numpy.linalg import inv
from scipy.stats import chi2
from pandas import read_csv
import statsmodels.api as sm
import os

In [3]:
os.getcwd()

'/Users/mandarphatak'

In [4]:
os.chdir("/Users/mandarphatak/Downloads/")

In [5]:
os.getcwd()

'/Users/mandarphatak/Downloads'

# Importing the data for vanguard and the famafrench factors
    

In [11]:
data=read_csv("vanguard_famafrench.csv")

In [14]:
data = data.dropna()

data

,date,rf,mkt_excess,smb,hml,Communications Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,2004-10-01,0.0011,0.0143,0.0015,-0.0022,0.025895,0.021216,-0.002137,-0.014099,-0.002740,-0.028816,0.010482,0.020669,-0.020334,0.037515,0.035496
1,2004-11-01,0.0015,0.0454,0.0374,0.0141,0.052653,0.053536,0.040117,0.075274,0.035721,0.024940,0.069020,0.056655,0.096553,0.039548,0.043742
2,2004-12-01,0.0016,0.0343,-0.0003,-0.0022,0.036160,0.052639,0.042354,-0.028016,0.041629,0.057599,0.030434,0.026588,0.011761,0.047553,0.028172
3,2005-01-01,0.0016,-0.0276,-0.0172,0.0206,-0.055373,-0.038796,0.011545,0.037397,-0.030477,-0.032816,-0.030419,-0.059924,-0.032094,-0.085588,0.010587
4,2005-02-01,0.0016,0.0189,-0.0057,0.0154,0.018897,-0.000566,0.005435,0.165533,-0.003067,0.030845,0.015217,0.005651,0.084211,0.031715,0.021799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023-07-01,0.0045,0.0321,0.0208,0.0411,0.059355,0.033969,0.022371,0.082913,0.060192,0.010293,0.031431,0.028610,0.039457,0.020345,0.024056
226,2023-08-01,0.0045,-0.0239,-0.0316,-0.0106,-0.028503,-0.020149,-0.031891,0.019468,-0.030535,-0.011725,-0.020520,-0.021833,-0.035263,-0.033779,-0.063397
227,2023-09-01,0.0043,-0.0524,-0.0251,0.0152,-0.035362,-0.058614,-0.045520,0.023748,-0.032034,-0.034630,-0.057756,-0.065384,-0.049823,-0.072638,-0.055950
228,2023-10-01,0.0047,-0.0319,-0.0387,0.0019,-0.015876,-0.051973,-0.017408,-0.053653,-0.029009,-0.039558,-0.039184,-0.017113,-0.038488,-0.036215,0.008154


# we can start with the analysis as per the methodology in the JH Cochrane (Asset Pricing)


In [15]:
# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['mkt_excess', 'smb', 'hml']].values
riskfree = data['rf'].values
portfolios = data.iloc[:, 5:].values

# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

# Shape information
T,K = factors.shape
T,N = portfolios.shape
# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree


In [16]:
# Time series regressions
X = sm.add_constant(factors)
ts_res = sm.OLS(excessReturns, X).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]
avgExcessReturns = mean(excessReturns, 0)
# Cross-section regression
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
riskPremia = cs_res.params

In [17]:
# Moment conditions
X = sm.add_constant(factors)
p = vstack((alpha, beta))
epsilon = excessReturns - X @ p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia[None,:] @ beta
moments2 = u * beta.T
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = (X.T @ X) / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta @ beta.T
for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp

vcv = inv(G.T) * S * inv(G) / T

In [18]:
# We increase by 4 as we are using three factors
vcvAlpha = vcv[0:N * K + N:4, 0:N * K + N:4]
J = alpha @ inv(vcvAlpha) @ alpha.T
J = J[0, 0]
Jpval = 1 - chi2(11).cdf(J)

In [19]:
print(J)
print(Jpval)

15.38425197168388
0.16557271766613857


In [ ]:
## We end up accepting the null for Vanguard Sectoral ETFs , alpha is equal to zero . 
#We proceed to calculate the annual risk premia and for individual sectoral funds

In [20]:
vcvRiskPremia = vcv[N * K + N:, N * K + N:]
annualizedRP = 12 * riskPremia
arp = list(squeeze(annualizedRP))
arpSE = list(sqrt(12 * diag(vcvRiskPremia)))
print('        Annualized Risk Premia')
print('           Market       SMB         HML ')
print('---------------------------------------------------------------')
print('Premia     {0:0.4f}    {1:0.4f}    {2:0.4f}   '.format(arp[0], arp[1], arp[2]))
print('Std. Err.  {0:0.4f}    {1:0.4f}     {2:0.4f}   '.format(arpSE[0], arpSE[1], arpSE[2]))
print('\n\n')

print('J-test:   {:0.4f}'.format(J))
print('P-value:   {:0.4f}'.format(Jpval))

i = 0
betaSE = []
for j in range(11):
    for k in range(1):
        a = alpha[i]
        b = beta[:, i]
        variances = diag(vcv[(K + 1) * i:(K + 1) * (i + 1), (K + 1) * i:(K + 1) * (i + 1)])
        betaSE.append(sqrt(variances))
        s = sqrt(variances)
        c = hstack((a, b))
        t = c / s
        print('Size: {:}, Value:{:}   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) '.format(j + 1, k + 1))
        print('Coefficients: {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f}'.format(a, b[0], b[1], b[2]))
        print('Std Err.      {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f}'.format(s[0], s[1], s[2], s[3]))
        print('T-stat        {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f}'.format(t[0], t[1], t[2], t[3]))
        print('')
        i += 1

        Annualized Risk Premia
           Market       SMB         HML 
---------------------------------------------------------------
Premia     0.0979    -0.0455    -0.0342   
Std. Err.  0.0105    0.0121     0.0096   



J-test:   15.3843
P-value:   0.1656
Size: 1, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) 
Coefficients:    -0.0015      0.8988      0.0007     -0.1223
Std Err.          0.0019      0.0522      0.0825      0.0640
T-stat           -0.7935     17.2117      0.0083     -1.9110

Size: 2, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) 
Coefficients:     0.0000      1.1697      0.2514     -0.0999
Std Err.          0.0013      0.0387      0.0638      0.0488
T-stat            0.0076     30.2268      3.9428     -2.0458

Size: 3, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) 
Coefficients:     0.0022      0.6360     -0.3717      0.0697
Std Err.          0.0015      0.0374      0.0600      0.0558
T-stat            1.5118     17.0154     -6.1927      1.2494

Si

In [ ]:
## We can clearly observe that for most Beta(MKT) dominates and is significant , for few Beta(SMB) is significant

In [ ]:
# We can also use the linearGMM package 

In [21]:
from linearmodels.asset_pricing import LinearFactorModelGMM
data = read_csv('vanguard_famafrench.csv')
data = data.dropna()
data

,date,rf,mkt_excess,smb,hml,Communications Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,2004-10-01,0.0011,0.0143,0.0015,-0.0022,0.025895,0.021216,-0.002137,-0.014099,-0.002740,-0.028816,0.010482,0.020669,-0.020334,0.037515,0.035496
1,2004-11-01,0.0015,0.0454,0.0374,0.0141,0.052653,0.053536,0.040117,0.075274,0.035721,0.024940,0.069020,0.056655,0.096553,0.039548,0.043742
2,2004-12-01,0.0016,0.0343,-0.0003,-0.0022,0.036160,0.052639,0.042354,-0.028016,0.041629,0.057599,0.030434,0.026588,0.011761,0.047553,0.028172
3,2005-01-01,0.0016,-0.0276,-0.0172,0.0206,-0.055373,-0.038796,0.011545,0.037397,-0.030477,-0.032816,-0.030419,-0.059924,-0.032094,-0.085588,0.010587
4,2005-02-01,0.0016,0.0189,-0.0057,0.0154,0.018897,-0.000566,0.005435,0.165533,-0.003067,0.030845,0.015217,0.005651,0.084211,0.031715,0.021799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023-07-01,0.0045,0.0321,0.0208,0.0411,0.059355,0.033969,0.022371,0.082913,0.060192,0.010293,0.031431,0.028610,0.039457,0.020345,0.024056
226,2023-08-01,0.0045,-0.0239,-0.0316,-0.0106,-0.028503,-0.020149,-0.031891,0.019468,-0.030535,-0.011725,-0.020520,-0.021833,-0.035263,-0.033779,-0.063397
227,2023-09-01,0.0043,-0.0524,-0.0251,0.0152,-0.035362,-0.058614,-0.045520,0.023748,-0.032034,-0.034630,-0.057756,-0.065384,-0.049823,-0.072638,-0.055950
228,2023-10-01,0.0047,-0.0319,-0.0387,0.0019,-0.015876,-0.051973,-0.017408,-0.053653,-0.029009,-0.039558,-0.039184,-0.017113,-0.038488,-0.036215,0.008154


In [22]:
data.iloc[:, 5:] = data.iloc[:, 5:].values - data[["rf"]].values



In [23]:
portfolios=data[["Communications Services","Consumer Discretionary", "Consumer Staples","Energy","Financials","Health Care","Industrials","Information Technology","Materials","Real Estate","Utilities"]]

In [24]:
portfolios

,Communications Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,0.024795,0.020116,-0.003237,-0.015199,-0.003840,-0.029916,0.009382,0.019569,-0.021434,0.036415,0.034396
1,0.051153,0.052036,0.038617,0.073774,0.034221,0.023440,0.067520,0.055155,0.095053,0.038048,0.042242
2,0.034560,0.051039,0.040754,-0.029616,0.040029,0.055999,0.028834,0.024988,0.010161,0.045953,0.026572
3,-0.056973,-0.040396,0.009945,0.035797,-0.032077,-0.034416,-0.032019,-0.061524,-0.033694,-0.087188,0.008987
4,0.017297,-0.002166,0.003835,0.163933,-0.004667,0.029245,0.013617,0.004051,0.082611,0.030115,0.020199
...,...,...,...,...,...,...,...,...,...,...,...
225,0.054855,0.029469,0.017871,0.078413,0.055692,0.005793,0.026931,0.024110,0.034957,0.015845,0.019556
226,-0.033003,-0.024649,-0.036391,0.014968,-0.035035,-0.016225,-0.025020,-0.026333,-0.039763,-0.038279,-0.067897
227,-0.039662,-0.062914,-0.049820,0.019448,-0.036334,-0.038930,-0.062056,-0.069684,-0.054123,-0.076938,-0.060250
228,-0.020576,-0.056673,-0.022108,-0.058353,-0.033709,-0.044258,-0.043884,-0.021813,-0.043188,-0.040915,0.003454


In [26]:
factors = data[["mkt_excess", "smb", "hml"]]
factors

,mkt_excess,smb,hml
0,0.0143,0.0015,-0.0022
1,0.0454,0.0374,0.0141
2,0.0343,-0.0003,-0.0022
3,-0.0276,-0.0172,0.0206
4,0.0189,-0.0057,0.0154
...,...,...,...
225,0.0321,0.0208,0.0411
226,-0.0239,-0.0316,-0.0106
227,-0.0524,-0.0251,0.0152
228,-0.0319,-0.0387,0.0019


In [27]:
mod = LinearFactorModelGMM(portfolios, factors)
res = mod.fit()
print(res)

Iteration: 0, Objective: 8.807299752887484
Iteration: 10, Objective: 6.7410227928000275
Iteration: 20, Objective: 6.481747376823608
Iteration: 30, Objective: 5.070715692105023
Iteration: 40, Objective: 4.909985144278462
         Current function value: 4.671359
         Iterations: 47
         Function evaluations: 3212
         Gradient evaluations: 80
Iteration: 0, Objective: 4.610502156690961
Iteration: 10, Objective: 4.6016645920696195
Iteration: 20, Objective: 4.601109157224402
Iteration: 30, Objective: 4.598610241116854
Iteration: 40, Objective: 4.5983245009551945
         Current function value: 4.598049
         Iterations: 44
         Function evaluations: 3332
         Gradient evaluations: 83
                    LinearFactorModelGMM Estimation Summary                     
No. Test Portfolios:                 11   R-squared:                      0.7191
No. Factors:                          3   J-statistic:                    4.5980
No. Observations:                   230   P-

In [28]:
# Now we may be having heteroscedasticity, so we go for  robustness
res = mod.fit(cov_type="kernel", kernel="bartlett", disp=0)
print(res)


                    LinearFactorModelGMM Estimation Summary                     
No. Test Portfolios:                 11   R-squared:                      0.7181
No. Factors:                          3   J-statistic:                    3.5803
No. Observations:                   230   P-value                         0.8929
Date:                  Sun, Jan 07 2024   Distribution:                  chi2(8)
Time:                          12:40:08                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
mkt_excess     0.0101     0.0023     4.4402     0.0000      0.0056      0.0145
smb           -0.0029     0.0018    

In [29]:
from scipy import linalg
from scipy.stats import f

In [30]:
res.full_summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                    LinearFactorModelGMM Estimation Summary                     
================================================================================
No. Test Portfolios:                 11   R-squared:                      0.7181
No. Factors:                          3   J-statistic:                    3.5803
No. Observations:                   230   P-value                         0.8929
Date:                  Sun, Jan 07 2024   Distribution:                  chi2(8)
Time:                          12:40:08                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
mkt_excess     0.0101     0.0023     4.4402     0.0000      0.0056      0.0145
smb           -0.0029     0.0018    -1.5903     0.1118     -0.0065      0.0007
hml           -0.0056     0.0021    -2.6589     0.0078     -0.0098     -0.0015


                     Communications Services Coefficients                     
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
alpha         -0.0021     0.0019    -1.1253     0.2605     -0.0057      0.0016
mkt_excess     0.8905     0.0013     681.17     0.0000      0.8879      0.8930
smb            0.0371     0.0013     27.570     0.0000      0.0345      0.0397
hml           -0.1035     0.0035    -29.290     0.0000     -0.1104     -0.0966


                     Consumer Discretionary Coefficients                      
==============================================================================
alpha      -9.849e-05     0.0013    -0.0763     0.9392     -0.0026      0.0024
mkt_excess     1.1838     0.0016     762.12     0.0000      1.1808      1.1869
smb            0.2381     0.0011     211.15     0.0000      0.2359      0.2403
hml           -0.0809     0.0012    -65.292     0.0000     -0.0834     -0.0785


                        Consumer Staples Coefficients                         
==============================================================================
alpha          0.0006     0.0017     0.3363     0.7367     -0.0028      0.0040
mkt_excess     0.6573     0.0028     237.90     0.0000      0.6519      0.6627
smb           -0.4005     0.0022    -181.09     0.0000     -0.4049     -0.3962
hml            0.0604     0.0425     1.4218     0.1551     -0.0229      0.1437


                             Energy Coefficients                              
==============================================================================
alpha          0.0028     0.0684     0.0403     0.9678     -0.1312      0.1367
mkt_excess     1.0865     0.0529     20.554     0.0000      0.9829      1.1901
smb            0.3649     0.0307     11.888     0.0000      0.3047      0.4250
hml            0.6668     0.0667     10.000     0.0000      0.5361      0.7975


                           Financials Coefficients                            
==============================================================================
alpha         -0.0008     0.0519    -0.0157     0.9875     -0.1025      0.1009
mkt_excess     1.0983     0.0226     48.556     0.0000      1.0539      1.1426
smb           -0.0089     0.0453    -0.1974     0.8435     -0.0977      0.0798
hml            0.7538     0.0409     18.443     0.0000      0.6737      0.8339


                           Health Care Coefficients                           
=============================================

In [31]:
def grs(res_output, N, K, factors):
    T = res_output.nobs
    N = N  # number of portfolios
    K = K  # number of factors
    # dividing the GRS equation
    a = (T - N - K) / N
    # omega hat should be a K x K matrix (verified and True)
    E_f = factors.mean()
    omega_hat = (1 / T) * (factors - E_f).T.dot(factors - E_f)
    # b should be a scalar (verified and True)
    omega_hat_inv = linalg.pinv(omega_hat)  # pseudo-inverse
    b = 1 + ((E_f.T).dot(omega_hat_inv).dot(E_f))
    b_inv = b**(-1)
    # Part c
    # sigma hat should be a N x N matrix (verified and True)
    sigma_hat = res_output.std_errors
    sigma_hat = (sigma_hat).dot(sigma_hat.T)
    sigma_hat_inv = linalg.pinv(sigma_hat)  # pseudo-inverse
    alpha_hat = res_output.alphas
    c = alpha_hat.dot(sigma_hat_inv).dot(alpha_hat.T)
    # Putting the 3 GRS parts together
    grs = a * b_inv * c
    print(grs)
    dfn = N
    dfd = T - N - K
    p_value = 1 - f.cdf(grs, dfn, dfd)
    print('p-value', p_value)


In [32]:
grs(res, 11, 3, factors)

0.01286918803668898
p-value 0.9999999982741129


# the GLS test (Gibbons Ross and Shanken Test):-Gibbons, Ross & Shanken (1989)  which tests whether the factors fully explain the expected returns of various portfolios, the test suggests that the three -factor model improves the explanatory power of the returns of stocks relative to the three-factor model in our case. 

In [33]:
## We proceed further to check for five facotr model

data=read_csv('vanguard_five_factors.csv')
data

,date,rf,mkt_excess,smb,hml,rmw,cma,VOX,VCR,VDC,VDE,VFH,VHT,VIS,VGT,VAW,VNQ,VPU
0,2004-10-01,0.0011,0.0143,0.0019,-0.0022,-0.0052,0.0049,0.025895,0.021216,-0.002137,-0.014098,-0.002740,-0.028816,0.010482,0.020669,-0.020334,0.037514,0.035496
1,2004-11-01,0.0015,0.0454,0.0411,0.0141,-0.0058,-0.0022,0.052653,0.053536,0.040117,0.075273,0.035721,0.024941,0.069020,0.056655,0.096553,0.039549,0.043742
2,2004-12-01,0.0016,0.0343,-0.0005,-0.0022,-0.0116,0.0046,0.036160,0.052639,0.042355,-0.028016,0.041629,0.057599,0.030434,0.026588,0.011761,0.047553,0.028173
3,2005-01-01,0.0016,-0.0276,-0.0117,0.0206,0.0274,-0.0146,-0.055374,-0.038796,0.011545,0.037397,-0.030478,-0.032816,-0.030419,-0.059924,-0.032094,-0.085588,0.010587
4,2005-02-01,0.0016,0.0189,-0.0031,0.0154,0.0143,-0.0005,0.018897,-0.000566,0.005435,0.165533,-0.003067,0.030845,0.015217,0.005651,0.084211,0.031715,0.021799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023-07-01,0.0045,0.0321,0.0286,0.0411,-0.0057,0.0057,0.059355,0.033969,0.022371,0.082913,0.060192,0.010293,0.031431,0.028610,0.039457,0.020345,0.024056
226,2023-08-01,0.0045,-0.0239,-0.0365,-0.0106,0.0343,-0.0237,-0.028503,-0.020149,-0.031891,0.019468,-0.030535,-0.011725,-0.020520,-0.021833,-0.035263,-0.033779,-0.063397
227,2023-09-01,0.0043,-0.0524,-0.0180,0.0152,0.0186,-0.0083,-0.035362,-0.058614,-0.045520,0.023748,-0.032034,-0.034630,-0.057756,-0.065384,-0.049823,-0.072638,-0.055950
228,2023-10-01,0.0047,-0.0319,-0.0404,0.0019,0.0246,-0.0066,-0.015876,-0.051973,-0.017408,-0.053653,-0.029009,-0.039558,-0.039184,-0.017113,-0.038488,-0.036215,0.008154


In [34]:
# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['mkt_excess', 'smb', 'hml','rmw','cma']].values
riskfree = data['rf'].values
portfolios = data.iloc[:, 7:].values

# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

# Shape information
T,K = factors.shape
T,N = portfolios.shape
# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree

In [48]:
# Time series regressions
X = sm.add_constant(factors)
ts_res = sm.OLS(excessReturns, X).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]
avgExcessReturns = mean(excessReturns, 0)
# Cross-section regression
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
riskPremia = cs_res.params

In [36]:
# Moment conditions
X = sm.add_constant(factors)
p = vstack((alpha, beta))
epsilon = excessReturns - X @ p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia[None,:] @ beta
moments2 = u * beta.T
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = (X.T @ X) / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta @ beta.T
for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp

vcv = inv(G.T) * S * inv(G) / T

In [37]:
##The 𝐽-test examines whether the average pricing errors, 𝛼̂ , are zero. The 𝐽statistic has an asymptotic 𝜒2𝑁 distribution
## Our model does not rejec the J-Test, alpha is equal to zero.
vcvAlpha = vcv[0:N * K + N:6, 0:N * K + N:6]
J = alpha @ inv(vcvAlpha) @ alpha.T
J = J[0, 0]
Jpval = 1 - chi2(11).cdf(J)
J
Jpval


0.45377005793069547

In [39]:
print(J)
print(Jpval)

10.875284091443952
0.45377005793069547


In [40]:
### We use the GRS test our results

factors = data[["mkt_excess", "smb", "hml","rmw","cma"]]
factors

portfolios = data[["VOX","VCR","VDC","VDE","VFH" ,"VHT" ,"VIS" ,"VGT" ,"VAW" ,"VNQ" ,"VPU" ]]

mod = LinearFactorModelGMM(portfolios, factors)
res = mod.fit(cov_type="kernel", kernel="bartlett", disp=0)


factors


,mkt_excess,smb,hml,rmw,cma
0,0.0143,0.0019,-0.0022,-0.0052,0.0049
1,0.0454,0.0411,0.0141,-0.0058,-0.0022
2,0.0343,-0.0005,-0.0022,-0.0116,0.0046
3,-0.0276,-0.0117,0.0206,0.0274,-0.0146
4,0.0189,-0.0031,0.0154,0.0143,-0.0005
...,...,...,...,...,...
225,0.0321,0.0286,0.0411,-0.0057,0.0057
226,-0.0239,-0.0365,-0.0106,0.0343,-0.0237
227,-0.0524,-0.0180,0.0152,0.0186,-0.0083
228,-0.0319,-0.0404,0.0019,0.0246,-0.0066


In [41]:
def grs(res_output, N, K, factors):
    T = res_output.nobs
    N = N  # number of portfolios
    K = K  # number of factors
    # dividing the GRS equation
    a = (T - N - K) / N
    # omega hat should be a K x K matrix (verified and True)
    E_f = factors.mean()
    omega_hat = (1 / T) * (factors - E_f).T.dot(factors - E_f)
    # b should be a scalar (verified and True)
    omega_hat_inv = linalg.pinv(omega_hat)  # pseudo-inverse
    b = 1 + ((E_f.T).dot(omega_hat_inv).dot(E_f))
    b_inv = b**(-1)
    # Part c
    # sigma hat should be a N x N matrix (verified and True)
    sigma_hat = res_output.std_errors
    sigma_hat = (sigma_hat).dot(sigma_hat.T)
    sigma_hat_inv = linalg.pinv(sigma_hat)  # pseudo-inverse
    alpha_hat = res_output.alphas
    c = alpha_hat.dot(sigma_hat_inv).dot(alpha_hat.T)
    # Putting the 3 GRS parts together
    grs = a * b_inv * c
    print(grs)
    dfn = N
    dfd = T - N - K
    p_value = 1 - f.cdf(grs, dfn, dfd)
    print('p-value', p_value)


In [42]:
grs(res,11,5,factors)

0.000898961710777145
p-value 0.9999999999999992


In [ ]:
### We performed the Gibbons , Ross and Shanken test, 
##where  Gibbons, Ross and Shanken (1989, GRS hereafter) developedand analyzed a test for the ex ante mean-variance efficiency of portfolios.

In [ ]:
## So now we can move forward for 6 factors ie using momentum

In [45]:
data = read_csv('vanguard_ff6_data.csv')

In [46]:
data

,date,rf,mkt_excess,smb,hml,rmw,cma,mom,Communications Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,2004-10-01,0.0011,0.0143,0.0019,-0.0022,-0.0052,0.0049,-0.0138,0.025895,0.021216,-0.002138,-0.014098,-0.002740,-0.028816,0.010482,0.020669,-0.020334,0.037514,0.035496
1,2004-11-01,0.0015,0.0454,0.0411,0.0141,-0.0058,-0.0022,0.0316,0.052652,0.053536,0.040117,0.075273,0.035721,0.024941,0.069021,0.056655,0.096553,0.039548,0.043743
2,2004-12-01,0.0016,0.0343,-0.0005,-0.0022,-0.0116,0.0046,-0.0287,0.036160,0.052639,0.042355,-0.028016,0.041629,0.057599,0.030434,0.026588,0.011761,0.047554,0.028172
3,2005-01-01,0.0016,-0.0276,-0.0117,0.0206,0.0274,-0.0146,0.0305,-0.055374,-0.038796,0.011544,0.037397,-0.030478,-0.032816,-0.030419,-0.059924,-0.032094,-0.085588,0.010587
4,2005-02-01,0.0016,0.0189,-0.0031,0.0154,0.0143,-0.0005,0.0337,0.018897,-0.000566,0.005435,0.165532,-0.003067,0.030845,0.015217,0.005651,0.084211,0.031714,0.021798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023-07-01,0.0045,0.0321,0.0286,0.0411,-0.0057,0.0057,-0.0398,0.059355,0.033969,0.022371,0.082913,0.060192,0.010293,0.031431,0.028610,0.039457,0.020345,0.024056
226,2023-08-01,0.0045,-0.0239,-0.0365,-0.0106,0.0343,-0.0237,0.0377,-0.028503,-0.020149,-0.031891,0.019468,-0.030535,-0.011725,-0.020520,-0.021833,-0.035263,-0.033779,-0.063397
227,2023-09-01,0.0043,-0.0524,-0.0180,0.0152,0.0186,-0.0083,0.0026,-0.035362,-0.058614,-0.045520,0.023748,-0.032034,-0.034630,-0.057756,-0.065384,-0.049823,-0.072638,-0.055950
228,2023-10-01,0.0047,-0.0319,-0.0404,0.0019,0.0246,-0.0066,0.0173,-0.015876,-0.051973,-0.017408,-0.053653,-0.029009,-0.039558,-0.039184,-0.017113,-0.038488,-0.036215,0.008154


In [47]:
# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['mkt_excess', 'smb', 'hml','rmw','cma','mom']].values
riskfree = data['rf'].values
portfolios = data.iloc[:, 8:].values

# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

# Shape information
T,K = factors.shape
T,N = portfolios.shape
# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree

In [49]:
# Time series regressions
X = sm.add_constant(factors)
ts_res = sm.OLS(excessReturns, X).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]
avgExcessReturns = mean(excessReturns, 0)
# Cross-section regression
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
riskPremia = cs_res.params

In [50]:
# Moment conditions
X = sm.add_constant(factors)
p = vstack((alpha, beta))
epsilon = excessReturns - X @ p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia[None,:] @ beta
moments2 = u * beta.T
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = (X.T @ X) / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta @ beta.T
for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp

vcv = inv(G.T) * S * inv(G) / T

In [51]:
##The 𝐽-test examines whether the average pricing errors, 𝛼̂ , are zero. The 𝐽statistic has an asymptotic 𝜒2𝑁 distribution
## Our model does not rejec the J-Test, alpha is equal to zero.
vcvAlpha = vcv[0:N * K + N:7, 0:N * K + N:7]
J = alpha @ inv(vcvAlpha) @ alpha.T
J = J[0, 0]
Jpval = 1 - chi2(11).cdf(J)




In [52]:
print(J)
print(Jpval)

10.146070825864145
0.5172978791041001


In [ ]:
## We again have the similar outcome that our model doesnt reject the J statistics, alpha is = zero ie no pricing erorr

In [54]:
data=read_csv('vanguard_ff6_data.csv')
data

,date,rf,mkt_excess,smb,hml,rmw,cma,mom,Communications Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,2004-10-01,0.0011,0.0143,0.0019,-0.0022,-0.0052,0.0049,-0.0138,0.025895,0.021216,-0.002138,-0.014098,-0.002740,-0.028816,0.010482,0.020669,-0.020334,0.037514,0.035496
1,2004-11-01,0.0015,0.0454,0.0411,0.0141,-0.0058,-0.0022,0.0316,0.052652,0.053536,0.040117,0.075273,0.035721,0.024941,0.069021,0.056655,0.096553,0.039548,0.043743
2,2004-12-01,0.0016,0.0343,-0.0005,-0.0022,-0.0116,0.0046,-0.0287,0.036160,0.052639,0.042355,-0.028016,0.041629,0.057599,0.030434,0.026588,0.011761,0.047554,0.028172
3,2005-01-01,0.0016,-0.0276,-0.0117,0.0206,0.0274,-0.0146,0.0305,-0.055374,-0.038796,0.011544,0.037397,-0.030478,-0.032816,-0.030419,-0.059924,-0.032094,-0.085588,0.010587
4,2005-02-01,0.0016,0.0189,-0.0031,0.0154,0.0143,-0.0005,0.0337,0.018897,-0.000566,0.005435,0.165532,-0.003067,0.030845,0.015217,0.005651,0.084211,0.031714,0.021798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023-07-01,0.0045,0.0321,0.0286,0.0411,-0.0057,0.0057,-0.0398,0.059355,0.033969,0.022371,0.082913,0.060192,0.010293,0.031431,0.028610,0.039457,0.020345,0.024056
226,2023-08-01,0.0045,-0.0239,-0.0365,-0.0106,0.0343,-0.0237,0.0377,-0.028503,-0.020149,-0.031891,0.019468,-0.030535,-0.011725,-0.020520,-0.021833,-0.035263,-0.033779,-0.063397
227,2023-09-01,0.0043,-0.0524,-0.0180,0.0152,0.0186,-0.0083,0.0026,-0.035362,-0.058614,-0.045520,0.023748,-0.032034,-0.034630,-0.057756,-0.065384,-0.049823,-0.072638,-0.055950
228,2023-10-01,0.0047,-0.0319,-0.0404,0.0019,0.0246,-0.0066,0.0173,-0.015876,-0.051973,-0.017408,-0.053653,-0.029009,-0.039558,-0.039184,-0.017113,-0.038488,-0.036215,0.008154


In [55]:
data.iloc[:, 8:] = data.iloc[:, 8:].values - data[["rf"]].values

In [59]:
## We can test the GRS test
### We use the GRS test our results

factors = data[["mkt_excess", "smb", "hml","rmw","cma",'mom']]


portfolios = data[["Communications Services","Consumer Discretionary", "Consumer Staples","Energy","Financials","Health Care","Industrials","Information Technology","Materials","Real Estate","Utilities" ]]

mod = LinearFactorModelGMM(portfolios, factors)
res = mod.fit(cov_type="kernel", kernel="bartlett", disp=0)



print(res)

                    LinearFactorModelGMM Estimation Summary                     
No. Test Portfolios:                 11   R-squared:                      0.7350
No. Factors:                          6   J-statistic:                    1.7531
No. Observations:                   230   P-value                         0.8821
Date:                  Sun, Jan 07 2024   Distribution:                  chi2(5)
Time:                          14:38:27                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
mkt_excess     0.0093     0.0027     3.4236     0.0006      0.0040      0.0146
smb           -0.0011     0.0031    

In [60]:
res.full_summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                    LinearFactorModelGMM Estimation Summary                     
================================================================================
No. Test Portfolios:                 11   R-squared:                      0.7350
No. Factors:                          6   J-statistic:                    1.7531
No. Observations:                   230   P-value                         0.8821
Date:                  Sun, Jan 07 2024   Distribution:                  chi2(5)
Time:                          14:38:27                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
mkt_excess     0.0093     0.0027     3.4236     0.0006      0.0040      0.0146
smb           -0.0011     0.0031    -0.3569     0.7212     -0.0071      0.0049
hml           -0.0036     0.0031    -1.1494     0.2504     -0.0098      0.0025
rmw            0.0017     0.0030     0.5622     0.5740     -0.0042      0.0075
cma           -0.0017     0.0033    -0.5020     0.6157     -0.0082      0.0048
mom            0.0112     0.0082     1.3646     0.1724     -0.0049      0.0273


                     Communications Services Coefficients                     
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
alpha         -0.0014     0.0018    -0.7528     0.4516     -0.0050      0.0022
mkt_excess     0.8676     0.0013     685.85     0.0000      0.8651      0.8701
smb            0.0799     0.0012     66.690     0.0000      0.0775      0.0822
hml           -0.2131     0.0035    -61.507     0.0000     -0.2199     -0.2063
rmw            0.2352     0.0011     213.79     0.0000      0.2331      0.2374
cma            0.1496     0.0015     99.996     0.0000      0.1467      0.1525
mom           -0.1056     0.0011    -96.617     0.0000     -0.1077     -0.1034


                     Consumer Discretionary Coefficients                      
==============================================================================
alpha          0.0003     0.0012     0.2206     0.8254     -0.0021      0.0027
mkt_excess     1.1378     0.0017     661.39     0.0000      1.1345      1.1412
smb            0.2285     0.0027     83.676     0.0000      0.2231      0.2338
hml           -0.1696     0.0021    -80.455     0.0000     -0.1737     -0.1654
rmw            0.1294     0.0393     3.2931     0.0010      0.0524      0.2064
cma           -0.0878     0.0820    -1.0699     0.2847     -0.2485      0.0730
mom           -0.1107     0.0866    -1.2786     0.2010     -0.2804      0.0590


                        Consumer Staples Coefficients                         
==============================================================================
alpha          0.0003     0.1035     0.0029     0.9977     -0.2026      0.2032
mkt_excess     0.7024     0.1308     5.3702     0.0000      0.4461      0.9588
smb           -0.2462     0.0595    -4.1379     0.0000     -0.3628     -0.1296
hml           -0.0893     0.0431    -2.0734     0.0381     -0.1737     -0.0049
rmw            0.3431     0.0674     5.0888     0.0000      0.2110      0.4753
cma            0.4830     0.0480     10.067     0.0000      0.3889      0.5770
mom            0.0206     0.0846     0.2438     0.8074     -0.1452      0.1864


                             Energy Coefficients                              
=================================================

In [61]:
def grs(res_output, N, K, factors):
    T = res_output.nobs
    N = N  # number of portfolios
    K = K  # number of factors
    # dividing the GRS equation
    a = (T - N - K) / N
    # omega hat should be a K x K matrix (verified and True)
    E_f = factors.mean()
    omega_hat = (1 / T) * (factors - E_f).T.dot(factors - E_f)
    # b should be a scalar (verified and True)
    omega_hat_inv = linalg.pinv(omega_hat)  # pseudo-inverse
    b = 1 + ((E_f.T).dot(omega_hat_inv).dot(E_f))
    b_inv = b**(-1)
    # Part c
    # sigma hat should be a N x N matrix (verified and True)
    sigma_hat = res_output.std_errors
    sigma_hat = (sigma_hat).dot(sigma_hat.T)
    sigma_hat_inv = linalg.pinv(sigma_hat)  # pseudo-inverse
    alpha_hat = res_output.alphas
    c = alpha_hat.dot(sigma_hat_inv).dot(alpha_hat.T)
    # Putting the 3 GRS parts together
    grs = a * b_inv * c
    print(grs)
    dfn = N
    dfd = T - N - K
    p_value = 1 - f.cdf(grs, dfn, dfd)
    print('p-value', p_value)


In [62]:
grs(res_output=res,N=11,K=6,factors=factors)

0.016284804034885823
p-value 0.9999999937955435


In [ ]:
###Our GRS test is also valid.